In [4]:
!git clone https://github.com/vicp5/llm-lnl

Cloning into 'llm-lnl'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 67 (delta 23), reused 56 (delta 12), pack-reused 0
Unpacking objects: 100% (67/67), 16.26 KiB | 925.00 KiB/s, done.


In [5]:
%cd llm-lnl/examples/embeddings

/content/llm-lnl/examples/embeddings


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [18]:
%env OPENAI_API_KEY=XXXXXXXXXXX

env: OPENAI_API_KEY=XXXXXXXXXXX


In [7]:
from wiki import tokenize_wikipedia_pages

In [8]:
df = tokenize_wikipedia_pages('./wc2022.csv', ['2022 FIFA World Cup'], ['2022 FIFA World Cup'])

recursively downloading pages
downloaded wikipedia page: 2022 FIFA World Cup
downloaded wikipedia page: 2018 and 2022 FIFA World Cup bids
downloaded wikipedia page: 2022 FIFA World Cup final
downloaded wikipedia page: 2022 FIFA World Cup officials
extracting tokenized sections


Token indices sequence length is longer than the specified maximum sequence length for this model (1192 > 1024). Running this sequence through the model will result in indexing errors


backing up csv to ./wc2022.csv


In [9]:
df.head()

,title,heading,content,tokens
0,2002 FIFA World Cup,Summary,"The 2002 FIFA World Cup, also branded as Korea...",411
1,2002 FIFA World Cup,Host selection,South Korea and Japan were selected as hosts b...,486
2,2002 FIFA World Cup,Qualification,199 teams attempted to qualify for the 2002 Wo...,424
3,2002 FIFA World Cup,Venues,"South Korea and Japan each provided 10 venues,...",112
4,2002 FIFA World Cup,Match officials,There was much controversy over the refereeing...,99


In [10]:
df = df.set_index(['title', 'heading'])

In [11]:
df.head()

content  \
title               heading                                                              
2002 FIFA World Cup Summary          The 2002 FIFA World Cup, also branded as Korea...   
                    Host selection   South Korea and Japan were selected as hosts b...   
                    Qualification    199 teams attempted to qualify for the 2002 Wo...   
                    Venues           South Korea and Japan each provided 10 venues,...   
                    Match officials  There was much controversy over the refereeing...   

                                     tokens  
title               heading                  
2002 FIFA World Cup Summary             411  
                    Host selection      486  
                    Qualification       424  
                    Venues              112  
                    Match officials      99

In [12]:
from embed import (
    compute_doc_embeddings,
    construct_prompt,
    order_document_sections_by_query_similarity,
    answer_query_with_context
)

In [13]:
docs = compute_doc_embeddings(df)

In [14]:
q = "who won the 2022 world cup?"

In [15]:
order_document_sections_by_query_similarity(q, docs)[:2]

[(0.8612033823202597, ('2022 FIFA World Cup final', 'Summary')),
 (0.8514635782933798, ('2022 FIFA World Cup final', 'Background'))]

In [16]:
print(construct_prompt(q, docs, df))

Selected 1 document sections:
('2022 FIFA World Cup final', 'Summary')
Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* The 2022 FIFA World Cup final was the final match of the 2022 FIFA World Cup, the 22nd edition of FIFA's competition for men's national football teams. The match was played at Lusail Stadium in Lusail, Qatar, on 18 December 2022, the Qatari National Day, and was contested by Argentina and defending champions France. The tournament comprised hosts Qatar and 31 other teams who emerged from the qualification phase, organised by the six FIFA confederations. The 32 teams competed in a group stage, from which 16 teams qualified for the knockout stage. En route to the final, Argentina finished first in Group C, with two wins and one loss, before defeating Australia in the round of 16, the Netherlands in the quarter-final through a penalty shoot-out and Croatia 

In [17]:
print(answer_query_with_context(q, df, docs))

Selected 1 document sections:
('2022 FIFA World Cup final', 'Summary')
Argentina won the 2022 World Cup.
